In [1]:
%cd ..
%load_ext autoreload
%autoreload 2

RTS_DATA = "/mnt/d/rts/"
METADATA = RTS_DATA + 'metadata'
VIDEOS = RTS_DATA + '0'
REMOTE_VIDEOS = "/mnt/rts"
OUTDIR = 'data'



/home/kikohs/work/rts


In [2]:
from datetime import datetime
import orjson
import pandas as pd
import zipfile
import os
import io
from typing import Dict, List, Optional, Tuple

In [4]:
# LOCAL imports
import rts

LOG = rts.utils.get_logger()

In [18]:
# vidx = rts.ingest.read_video_folder_index(os.path.join(OUTDIR, 'vidx.json'))
# df = rts.ingest.read_all_metadata(METADATA, vidx)
# rts.ingest.metadata_to_hdf5(OUTDIR, 'rts_metadata', df)

df = rts.ingest.load_metadata_hdf5(OUTDIR, 'rts_metadata')

In [103]:
client = rts.db.meilisearch.get_client()
midx = rts.db.meilisearch.get_or_create_media_index(client)
midx.get_stats().number_of_documents

0

In [8]:
#TODO get unique geo, people, for the whole media

{'taskUid': 1,
 'indexUid': 'books',
 'status': 'enqueued',
 'type': 'indexDeletion',
 'enqueuedAt': '2022-12-12T10:52:57.477125004Z'}

In [100]:
midx.delete_all_documents()

TaskInfo(task_uid=1910, index_uid='media', status='enqueued', type='documentDeletion', enqueued_at='2022-12-12T17:47:57.457834798Z')

In [104]:
rts.db.meilisearch.index_media(df, midx)

In [39]:
import whisper

model = whisper.load_model("base")
# result = model.transcribe("audio.mp3")
# print(result["text"])

/home/kikohs/.cache/pypoetry/virtualenvs/rts-DIiCth0j-py3.8/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [68]:
df.publishedBy.apply(lambda x: len(x) if x else 0).unique()

array([1, 0])

In [95]:
# midx = rts.db.meilisearch.get_or_create_media_index(client)
midx.get_stats().number_of_documents

135749

In [101]:
client.get_tasks()

{'results': [{'uid': 1910,
   'indexUid': 'media',
   'status': 'enqueued',
   'type': 'documentDeletion',
   'canceledBy': None,
   'details': {'deletedDocuments': None},
   'error': None,
   'duration': None,
   'enqueuedAt': '2022-12-12T17:47:57.457834798Z',
   'startedAt': None,
   'finishedAt': None},
  {'uid': 1909,
   'indexUid': 'media',
   'status': 'enqueued',
   'type': 'settingsUpdate',
   'canceledBy': None,
   'details': {'displayedAttributes': ['guid',
     'mediaId',
     'title',
     'resume',
     'categoryName',
     'assetType',
     'contentType',
     'backgoundType',
     'collection',
     'publishedDate'],
    'searchableAttributes': ['mediaId',
     'categoryName',
     'assetType',
     'contentType',
     'backgroundType',
     'collection',
     'title',
     'resume',
     'published'],
    'filterableAttributes': ['assetType',
     'backgoundType',
     'categoryName',
     'collection',
     'contentType',
     'publishedDate'],
    'sortableAttributes'